In [ ]:
import numpy as np
import scipy
import time

In [ ]:
def gaussian(x, parameter):

  heavy_computation = np.random.uniform(0,0.01)
  time.sleep(heavy_computation)

  return np.exp(-(x* parameter)**2)

integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(np.random.randint(1,100)))

integral

0.07089815403622066

In [ ]:
start_time = time.time()

integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(np.random.randint(1,100)))

print('That took ', time.time()-start_time, 'sec ...')

That took  2.042951822280884 sec ...


In [ ]:
num_proc = 5

start_time = time.time()

for i in range(5):
  integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(np.random.randint(1,100)))

print('That took ', time.time()-start_time, 'sec ...')

That took  8.24123239517212 sec ...


In [ ]:
import multiprocessing

In [ ]:
shared_memory = multiprocessing.Array('f', range(5))

def integrate(gaussian, lower_limit, upper_limit, i, shared_memory):

  start_time = time.time()

  integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(np.random.randint(1,100)))

  shared_memory[i] = integral

  print('The value of ', i, 'th integral is', integral)

  #print(i, ' took ', time.time()-start_time, 'sec ...')

In [ ]:
start_time = time.time()

for i in range(num_proc):
  integrate(gaussian, -np.inf, np.inf, i, shared_memory)

print('That took ', time.time()-start_time, 'sec ...')

shared_memory[:]

The value of  0 th integral is 0.03544907701811032
The value of  1 th integral is 0.01926580272723387
The value of  2 th integral is 0.02395207906629077
The value of  3 th integral is 0.025687736969645165
The value of  4 th integral is 0.02532076929865023
That took  8.824102640151978 sec ...


[0.0354490764439106,
 0.019265802577137947,
 0.023952079936861992,
 0.02568773739039898,
 0.02532077021896839]

In [ ]:
#Resetting shared memory
shared_memory = multiprocessing.Array('f', range(5))

shared_memory[:]

[0.0, 1.0, 2.0, 3.0, 4.0]

In [ ]:
i = 0
process = multiprocessing.Process(target=integrate, args=(gaussian,-np.inf, np.inf, i, shared_memory ))
process.start()

i=1
process_2 = multiprocessing.Process(target=integrate, args=(gaussian,-np.inf, np.inf, i, shared_memory ))
process_2.start()

process.join()
process_2.join()

shared_memory[:]

The value of  0 th integral is 0.25320769298650225The value of 
 1 th integral is 0.25320769298650225


[0.2532076835632324, 0.2532076835632324, 2.0, 3.0, 4.0]

In [ ]:
process_array=[]

start_time = time.time()

for i in range(num_proc):
  process = multiprocessing.Process(target=integrate, args=(gaussian,-np.inf, np.inf, i, shared_memory ))
  process.start()
  process_array.append(process)

for proc in process_array:
  proc.join()

print('That took ', time.time()-start_time, 'sec ...')

shared_memory[:]

The value of  0 The value of  th integral is1  th integral isThe value of The value of 0.25320769298650225  3 0.25320769298650225 2
 th integral isThe value of  th integral is
 0.25320769298650225 0.253207692986502254

 th integral is 0.25320769298650225
That took  1.5291125774383545 sec ...


[0.2532076835632324,
 0.2532076835632324,
 0.2532076835632324,
 0.2532076835632324,
 0.2532076835632324]

In [ ]:
def integrate_pipe(gaussian, lower_limit, upper_limit, i, send_end, parameter):

  start_time = time.time()

  integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(parameter))

  send_end.send([i, integral])

  #print('The value of ', i, 'th integral is', integral)

In [ ]:
rec_end, send_end = multiprocessing.Pipe()

start_time = time.time()

for i in range(num_proc):

  parameter = np.random.randint(1,100)
  process = multiprocessing.Process(target=integrate_pipe, args=(gaussian,-np.inf, np.inf, i, send_end, parameter))

  process.start()

  integral = rec_end.recv()

  #print('value of integral is', integral, '\n')

print('That took ', time.time()-start_time, 'sec ...')

That took  8.32703447341919 sec ...


In [ ]:
def integrate_map(agruments):

  gaussian, lower_limit, upper_limit, i, parameter = agruments

  start_time = time.time()

  integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(parameter))

  #print('The value of ', i, 'th integral is', integral)
  print(i, 'took', time.time()-start_time, 'secs ...')

  return [i, integral]

In [ ]:
input = []
for i in range(num_proc):
    parameter = np.random.randint(1,100)
    input.append([gaussian,-np.inf, np.inf, i, parameter])

start_time = time.time()

with multiprocessing.Pool() as pool:

  for integral in pool.map(integrate_map, input):

    print('integrate value', integral)

print('That took ', time.time()-start_time, 'sec ...')

0 took 1.0274367332458496 secs ...
1 took 1.369034767150879 secs ...
23  tooktook  1.698734998703003 2.0411696434020996secs ... 
secs ...
4 took 1.6962239742279053 secs ...
integrate value [0, 0.44311346272637897]
integrate value [1, 0.06330192324662558]
integrate value [2, 0.02188214630747551]
integrate value [3, 0.023321761196125227]
integrate value [4, 0.029056620506647802]
That took  4.990696907043457 sec ...


In [ ]:
import concurrent.futures

In [ ]:
with concurrent.futures.ProcessPoolExecutor() as executor:

  parameter = np.random.randint(1,100)

  proc = executor.submit(scipy.integrate.quad, gaussian, -np.inf, np.inf, args=(parameter))

  print(proc.result())

(0.02243612469500655, 1.1769086727846437e-08)


In [ ]:
list_a = []
for a in range(10):
  list_a.append(a)

list_b = [a for a in range(10)]

list_a, list_b

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
start_time = time.time()

with concurrent.futures.ProcessPoolExecutor() as executor:

  proc = [executor.submit(scipy.integrate.quad, gaussian, -np.inf, np.inf, args=(parameter)) for parameter in np.random.randint(1,100, size=num_proc)]

  for proc in concurrent.futures.as_completed(proc):
    print(proc.result())

print('That took ', time.time()-start_time, 'sec ...')

(0.029056620506647802, 4.749048476451983e-10)
(0.02496413874514812, 8.870637987178076e-10)
(0.2215567313631895, 1.414810651274887e-12)
(0.018463060946932456, 3.1559970731440157e-12)
(0.02395207906629077, 2.6712393762337756e-09)
That took  4.917762756347656 sec ...


In [ ]:
def integrate_concurrent_map(argument):

  gaussian, lower_limit, upper_limit, i, parameter = argument

  integral, error = scipy.integrate.quad(gaussian, -np.inf, np.inf, args=(parameter))

  return [i, integral]

In [ ]:
input = [[gaussian, -np.inf, np.inf, i, parameter] for i, parameter in enumerate(np.random.randint(1,100, size=num_proc))]

start_time = time.time()

with concurrent.futures.ProcessPoolExecutor() as executor:

    proc = executor.map(integrate_concurrent_map, input)

    for result in proc:
      print(result)

print('That took ', time.time()-start_time, 'sec ...')

[0, 0.5908179503018385]
[1, 0.2215567313631895]
[2, 0.029540897515091934]
[3, 0.024280189738431733]
[4, 0.03853160545446774]
That took  4.510158061981201 sec ...


In [ ]:
char_list = ['a', 'b', 'c', 'd']

i = 0
for character in char_list:
  print(i, character)
  i += 1

print('')

for i, character in enumerate(char_list):
  print(i, character)

0 a
1 b
2 c
3 d

0 a
1 b
2 c
3 d


In [ ]:
# Pickle File

In [ ]:
import pickle
import numpy as np

In [ ]:
item = np.arange(10000).reshape((100, 100))

In [ ]:
item, item.shape

(array([[   0,    1,    2, ...,   97,   98,   99],
        [ 100,  101,  102, ...,  197,  198,  199],
        [ 200,  201,  202, ...,  297,  298,  299],
        ...,
        [9700, 9701, 9702, ..., 9797, 9798, 9799],
        [9800, 9801, 9802, ..., 9897, 9898, 9899],
        [9900, 9901, 9902, ..., 9997, 9998, 9999]]),
 (100, 100))

In [ ]:
# Pickle files are not human readable

In [ ]:
with open('pickleFile.pickle', 'wb') as file:
    pickle.dump(item, file)

In [ ]:
with open('pickleFile.pickle', 'rb') as file_open:
    itemNew = pickle.load(file_open)

In [ ]:
itemNew

array([[   0,    1,    2, ...,   97,   98,   99],
       [ 100,  101,  102, ...,  197,  198,  199],
       [ 200,  201,  202, ...,  297,  298,  299],
       ...,
       [9700, 9701, 9702, ..., 9797, 9798, 9799],
       [9800, 9801, 9802, ..., 9897, 9898, 9899],
       [9900, 9901, 9902, ..., 9997, 9998, 9999]])

In [ ]:
if (item == itemNew).all():
    print('this is the same file')

this is the same file


In [ ]:
# how to send multiple objects/ functions and etc.

In [ ]:
def open_pickle(fileName):
    with open(fileName, 'rb') as file:   # reading from a pickle file
        obj1 = pickle.load(file)    # json.load()
        obj2 = pickle.load(file)
        #obj3 = pickle.load(file)
        #obj4 = pickle.load(file)
    return obj1, obj2#, obj3, obj4

def close_pickle(fileName, listValue):     # writing to a pickle file
    with open(fileName, 'wb') as file:
        pickle.dump(listValue[0], file)
        pickle.dump(listValue[1], file)
        #pickle.dump(listValue[2], file)
        #pickle.dump(listValue[3], file)
        #... inf
    return

In [ ]:
item1, item2, item3, item4 = item, item*10, item/10, item*0

In [ ]:
close_pickle('pickleFile.pickle', [ item1, item2, item3, item4 ])

In [ ]:
item11, item12, item13, item14  = open_pickle('pickleFile.pickle')

In [ ]:
item11

In [ ]:
# sending functions

def multiply(x, y, z, zq):
    return x * y * z * zq

In [ ]:
close_pickle('pickleFile.pickle', (multiply, item14))

In [ ]:
func_something, whatever_item =  open_pickle('pickleFile.pickle')

In [ ]:
whatever_item, func_something

In [ ]:
newObjItems = func_something(item11, item12, item13, item13)

In [ ]:
newObjItems_multiply = multiply(item11, item12, item13, item13)

In [ ]:
newObjItems_multiply == newObjItems